In [1]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

from importlib import reload  # Python 3.4+

import pickle

from pygents.tokens import Tokenizer 
from pygents.text import preprocess_text, grams_count_with_char_freedoms
from pygents.text import counters_init


In [2]:
assert str(Tokenizer().tokenize("ab c")) == "['ab', 'c']"

In [4]:

#TODO to utils

#TODO metrics on tokens, F1

def count_subelements(element):
    count = 0
    if isinstance(element, list) or isinstance(element, tuple):
        for child in element:
            count += count_subelements(child)
    elif isinstance(element, dict):
        for key in element:
            count += count_subelements(element[key])
    else:
        count = 1
    return count 
assert count_subelements(['1',2,[[3,'4',{'x':['5',6],'y':(7,'8')},{'z':{'p':9,'q':['10']}}]]]) == 10


class FreedomTokenizer(Tokenizer):

    def __init__(self, name=None, max_n=7, debug=False):
        Tokenizer.__init__(self,debug=debug)
        self.max_n = max_n
        self.model = pickle.load(open(name, 'rb')) if name is not None else [{},{},{}] 

    def train(self,texts,max_n=None):
        if max_n is None:
            max_n = self.max_n
        model = counters_init(max_n) 
        for text in texts:
            text = preprocess_text(text)
            chars = list(text)
            for n in range(max_n):
                grams_count_with_char_freedoms(model[0],model[1],model[2],chars,n+1,debug=self.debug)
        #merge n-specific models into joint ones
        for i in range(3):
            for d in model[i]:
                self.model[i].update(d)
        return self
        
    def tokenize(self,text):
        #TODO
        return text.split()

    def count_params(self):
        return count_subelements(self.model)
    
#TODO unittest "pig","pig"/"ding","dong"

_test_tokenizer = FreedomTokenizer(max_n=2,debug=True).train(["pig"])
assert _test_tokenizer.count_params() == 11
print(str(_test_tokenizer.model[1]))
print(str(_test_tokenizer.model[2]))
#assert str(_test_tokenizer.model) == "[{'p': 1, 'i': 1, 'g': 1, 'pi': 1, 'ig': 1}, {'p': {'g': 1}, 'i': {'g': 1}, 'pi': {'g': 1}}, {'i': {'p': 1}, 'g': {'i': 1}, 'ig': {'p': 1}}]"

#_test_tokenizer = FreedomTokenizer(max_n=2).train(["ding","dong"])

#print(str(_test_tokenizer.model[1]))
#print(str(_test_tokenizer.model[2]))
#print(str(_test_tokenizer.model))

#print( _test_tokenizer.count_params() ) #== 11



p
+ p i
i
+ i g
- i p
g
- g i
pi
+ pi g
ig
- ig p
{'p': {'i': 1}, 'i': {'g': 1}, 'pi': {'g': 1}}
{'i': {'p': 1}, 'g': {'i': 1}, 'ig': {'p': 1}}
